### Edge density constrained density minimisation

This note computes
$$
  \min_G t(C_5, G) \text{ w.r.t. }t(e, G) = p.
$$

Note that this is known when $p = 1 - 1 / k$ for $k=2, 3, 4, \ldots$ by [https://arxiv.org/abs/1803.00165] and the value in-between these densities are conjectured in the same paper.

In [ ]:
import flag_algebra.flag_algebra as fa
import flag_algebra.build_problem as bp
import networkx as nx
import cvxpy as cp

In [ ]:
# Five-cycle density minimisation with edge density constraint 1-1/n

H = nx.cycle_graph(5)
n = 3
k = 1

# Get the graph atlas
atlas = fa.get_graph_atlas(2 * n - k)

# Get H's density coefficients
h_coeffs = fa.compute_hom_coefficients(H, atlas)

# Get edge's density coefficients
edge = nx.Graph()
edge.add_edge(0, 1)
edge_coeffs = fa.compute_hom_coefficients(edge, atlas)

# Set up the SDP coefficients for n vertices and k labelled vertices
grouped_sdp_coeffs = fa.compute_grouped_averaged_flag_product_coefficients(atlas, n, k, verbose=True)

In [ ]:
# This code computes density of H in the complete k-partite graph, known as Turan density.
def compute_turan_density(H, k):
  m = H.number_of_nodes()

  chromatic_poly = nx.algorithms.polynomials.chromatic_polynomial(H)

  p_H_k = chromatic_poly.evalf(subs={'x': k})
  
  denominator = k ** m

  return p_H_k / denominator

In [ ]:
for p in range(3, 7):
  problem, variable_dict = bp.build_problem(
    objectives=[('hom', H, 1, h_coeffs)], # Density of H is objective
    constraints=[('hom', edge, 1 - 1/p, edge_coeffs)], # Edge density equals to 1 - 1 / p
    sdp_configs=[(n, k, grouped_sdp_coeffs)], # Use SDP matrix whose basis is formed by n-vertex flags with k labelled vertices
    lowerbound=True, # Solve for lower bound finding
  )
  problem.solve(solver=cp.SCS) # Solve with SCS solver. MOSEK can be used, but requires larger memory usage.

  print(f"Minimum density of C5 with edge density {1 - 1/p}: {problem.value}")
  print(f"Turan density of C5 for {p} colors: {compute_turan_density(H, p)}")